In [3]:
import pandas as pd
import numpy as np
import pickle
import re

In [35]:
# 2. Carreguem dades
df = pd.read_csv("data/datosFinal.csv")

In [ ]:
df

### Preprocessing

1. Company

In [36]:
condicio = (
    df["Company"].str.strip().str.upper().str.endswith("S.L") |
    df["Company"].str.strip().str.upper().str.endswith("SL") |
    df["Company"].str.strip().str.upper().str.endswith("S.L.") |
    df["Company"].str.upper().str.contains("SOCIEDAD LIMITADA")
)

df['es_sociedad_limitada'] = np.where(condicio, 'Sí', 'No')

df.es_sociedad_limitada.value_counts()

es_sociedad_limitada
Sí    544
No     79
Name: count, dtype: int64

2. Names

In [32]:
(df["Company"] == df["Name"]).value_counts()

True    623
Name: count, dtype: int64

3. Category

3. Investment

In [51]:
print(df.columns)
print(df.capital_prev)
print((
    df["capital_prev"]
    .astype(str)
    .str.extract(r'([\d.,]+M\s?€)')[0]   # agafa només la part numèrica amb M i €
    .str.replace("€", "", regex=False)
    .str.replace("M", "", regex=False)
    .str.replace("m", "", regex=False)
    .str.replace(",", ".", regex=False)       # canvia coma per punt decimal
    .str.replace(" ", "", regex=False)        # elimina espais
)
)

Index(['Startup', 'Company', 'Category', 'Investment', 'Investors', 'Date',
       'URL', 'Name', 'Description', 'Address', 'Industries', 'Technologies',
       'Other fields', 'Funding stage', 'Founded', 'Employees',
       'Business model', 'Target', 'Spinoff participants', 'capital_prev',
       'investors', 'es_sociedad_limitada'],
      dtype='object')
0        190M € (JANUARY 2025)
1         95M € (JANUARY 2024)
2      241.9M € (JANUARY 2022)
3          132M € (APRIL 2021)
4            53M € (JULY 2019)
                ...           
618                 (MAY 2022)
619            (DECEMBER 2020)
620               (MARCH 2022)
621             (JANUARY 2022)
622             (JANUARY 2022)
Name: capital_prev, Length: 623, dtype: object
0        190
1         95
2      241.9
3        132
4         53
       ...  
618      NaN
619      NaN
620      NaN
621      NaN
622      NaN
Name: 0, Length: 623, dtype: object


In [56]:
df["Investment"] = (
    df["capital_prev"]
    .astype(str)
    .str.extract(r'([\d.,]+M\s?€)')[0]   # agafa només la part numèrica amb M i €
    .str.replace("€", "", regex=False)
    .str.replace("M", "", regex=False)
    .str.replace("m", "", regex=False)
    .str.replace(",", ".", regex=False)       # canvia coma per punt decimal
    .str.replace(" ", "", regex=False)        # elimina espais
)

# 2. Conversió a numèric (float), posant NaN si hi ha errors
df["Investment"] = pd.to_numeric(df["Investment"], errors="coerce")

# 3. Substitueix 0 per NaN
df["Investment"] = df["Investment"].replace(0.0, np.nan)

# per tenir 0 en comptes de nan
#df["Investment"] = df["Investment"].fillna(0)


df.Investment

0      190.0
1       95.0
2      241.9
3      132.0
4       53.0
       ...  
618      NaN
619      NaN
620      NaN
621      NaN
622      NaN
Name: Investment, Length: 623, dtype: float64

4. Investor

In [ ]:
df["Investors"].value_counts()

In [ ]:
## OJO AQUÍ: SE ha de trabajar con la unión de los dos lados y con las dos variables. Posteriormente
## eliminar una variable. 

df.loc[(df["Investors"] != df["investors"]), ["Investors", "investors"]]


5. Date

In [38]:
df["Mes Inversion"] = [mes.split(" ")[0] for mes in df["Date"]]
df["Year Inversion"] = [mes.split(" ")[1] for mes in df["Date"]]

## Eliminar variable Date

6. Address

Sacamos las coordenadas a partir de la dirección

In [97]:
def limpiar_direccion(dir_raw):
    dir_limpia = dir_raw
    
    # Elimina paréntesis y su contenido
    dir_limpia = re.sub(r"\(.*?\)", "", dir_limpia)
    
    # Elimina referencias tipo P., Pta., Planta, Puerta, Loc, Sector, Sobreático, Num, Tienda
    dir_limpia = re.sub(r"\b(P\. |Pta\.?|Planta|Puerta|Loc|Sector|Sobreático|Num|Tienda)\b\.?\s*\d*\w*", "", dir_limpia, flags=re.IGNORECASE)
    
    # Elimina referencias tipo P., Pta., Planta, Puerta, Loc, Sector, Sobreático, Num, Tienda
    dir_limpia = re.sub(r"C/", "Carrer", dir_limpia, flags=re.IGNORECASE)
    dir_limpia = re.sub(r"Urbanizacion", "Urbanizacio", dir_limpia, flags=re.IGNORECASE)
    dir_limpia = re.sub(r"Rbla", "Rambla", dir_limpia, flags=re.IGNORECASE)
    dir_limpia = re.sub(r"Plaza", "Plaça", dir_limpia, flags=re.IGNORECASE)
    dir_limpia = re.sub(r"Pasaje", "Passatge", dir_limpia, flags=re.IGNORECASE)
    dir_limpia = re.sub(r"Paseig", "Passeig", dir_limpia, flags=re.IGNORECASE)
    
    # Elimina códigos postales (5 dígitos)
    dir_limpia = re.sub(r"\b\d{5}\b", "", dir_limpia)
    
    # Elimina comas innecesarias y múltiples espacios
    dir_limpia = re.sub(r",", "", dir_limpia)
    dir_limpia = re.sub(r"\s+", " ", dir_limpia)
    
    # Limpieza final de espacios
    dir_limpia = dir_limpia.strip()
    
    return dir_limpia

In [63]:
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import time

# Inicializa el geolocalizador
geolocator = Nominatim(user_agent="GetLoc")

# Función segura para geocodificar
def safe_geocode(direccion):
    try:
        location = geolocator.geocode(direccion, timeout=10)
        if location:
            latituds = location.latitude
            longituds = location.longitude
        else:
            print("No se encontró la dirección.")
            latituds = np.nan
            longituds = np.nan
    except GeocoderTimedOut:
        print("Timeout — intentando de nuevo...")
        time.sleep(1)
        return safe_geocode(direccion)
    except Exception as e:
        print("Error:", e)
    return (latituds, longituds)

In [86]:
direcciones = [re.sub("Calle", "Carrer", direc) for direc in df["Address"]]

In [ ]:
direcciones_limpias = [limpiar_direccion(d) for d in direcciones]
direcciones_limpias

In [ ]:
# Llama a la función con tu dirección
latituds = [safe_geocode(dir)[0] for dir in direcciones]

In [ ]:
direcciones

In [ ]:
latituds

5. Industries

In [ ]:
df["Industries"].value_counts()

In [ ]:
## OJO PORQUE HACIENDO ESTO SE DETECTA QUE HAY VARIABLES IGUALES ESCRITAS DIFERENTE (HEAKTHTECH)

dummies_Industries = df["Industries"].str.get_dummies(sep=',')
dummies_Industries = dummies_Industries.add_prefix('industr_')

# Combinar con el DataFrame original
df = df.join(dummies_Industries)

,\r\n\t\t \t\t\r\n\t\t \t\t\tAgriculture,\r\n\t\t \t\t\r\n\t\t \t\t\tBusiness Services & Software,\r\n\t\t \t\t\r\n\t\t \t\t\tE-commerce & Marketplaces,\r\n\t\t \t\t\r\n\t\t \t\t\tFinancing,\r\n\t\t \t\t\r\n\t\t \t\t\tFood,\r\n\t\t \t\t\r\n\t\t \t\t\tHealthtech,\r\n\t\t \t\t\r\n\t\t \t\t\tICT & Mobile,\r\n\t\t \t\t\r\n\t\t \t\t\tLegaltech,\r\n\t\t \t\t\r\n\t\t \t\t\tLogistic Tech,Business Services & Software,Community & Social network,E-commerce & Marketplaces,Fashion & Design,Food,Hardware,Healthtech,Proptech,Sports,Traveltech & Leisure
0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
88,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
89,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
90,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0


6. Technologies

In [ ]:
df["Technologies"].value_counts()

Technologies
E-commerce                                                                                                                                                                          20
IoT & Sensors                                                                                                                                                                       16
Personalized medicine, \r\n\t\t                \t\t\r\n\t\t                \t\t\tAdditive manufacturing, \r\n\t\t                \t\t\r\n\t\t                \t\t\tBiotechnology     8
Fintech & Insurtech, \r\n\t\t                \t\t\r\n\t\t                \t\t\tDigital Assets                                                                                        4
AI & Big Data, \r\n\t\t                \t\t\r\n\t\t                \t\t\tConnectivity, \r\n\t\t                \t\t\r\n\t\t                \t\t\tAutomation                          4
Digital Assets                                                          

In [ ]:
dummies_Technologies = df["Technologies"].str.get_dummies(sep=',')
dummies_Technologies = dummies_Technologies.add_prefix('tecno_')

# Combinar con el DataFrame original
df = df.join(dummies_Technologies)

7. Other fields

In [22]:
df["Other fields"].value_counts()

Other fields
Social Economy      36
Industry 4.0         4
Deeptech             4
Circular Economy     4
Sharing Economy      4
Smart Cities         4
Name: count, dtype: int64

In [ ]:
df["Other fields"] = df["Other fields"].replace("-", np.nan)
dum_of = df["Other fields"].str.get_dummies(sep=',')
dum_of = dum_of.add_prefix('tecno_')

# Combinar con el DataFrame original
df = df.join(dum_of)

8. Funding stage

In [23]:
df["Funding stage"].value_counts()

Funding stage
Seed (0.25 M EUR - < 1 M EUR)    60
Pre-seed (< 0.25 M EUR)          32
Name: count, dtype: int64

9. Founded

In [26]:
df["Founded"] = df["Founded"].astype(float)

10. Employees

In [27]:
df["Employees"].value_counts()

Employees
Undisclosed      48
From 1 to 5      20
From 6 to 10     16
From 11 to 20     4
From 21 to 50     4
Name: count, dtype: int64

11. Business model

In [ ]:
df["Business model"].value_counts()
df["Business model"] = df["Business model"].replace("-", np.nan)
dum_of = df["Business model"].str.get_dummies(sep=',')
dum_of = dum_of.add_prefix('businessModel_')


Business model
Saas                                                                            24
Development & Manufacturing                                                     20
-                                                                               16
Marketplace                                                                      8
Others                                                                           8
Ecommerce & Trade, \nSubscription                                                4
R+D+I Services                                                                   4
Ecommerce & Trade, \nMarketplace, \nSaas, \nSoftware licence, \nSubscription     4
Ecommerce & Trade                                                                4
Name: count, dtype: int64

12. Target

In [29]:
df["Target"].value_counts()

Target
Business                36
Consumer                28
-                       16
Business, \nConsumer    12
Name: count, dtype: int64

In [ ]:
df["Target"] = df["Target"].replace("-", np.nan)
dum_of = df["Target"].str.get_dummies(sep=',')
dum_of = dum_of.add_prefix('target_')

13. Spinoff participants

In [30]:
df["Spinoff participants"].value_counts()

Spinoff participants
None / Not a spinoff                                        76
Technological Centre (BIOCAT)                                8
CERCA Centre (CERCA Centre); \nCSIC Centre (CSIC Centre)     4
Company /Other Centre                                        4
Name: count, dtype: int64

Para la variable capita_prev contrastar con las variables Investor y Date si son iguales sino se deberá de trabajar para tener 2 variables independientes. 

In [35]:
df = df.drop(columns=["URL", "Date", "Name"])

## Guardamos la base de datos 

In [39]:
df.to_pickle(path="data/datos_preprocesados.pkl")
df.to_csv(path_or_buf="data/datos_preprocesados.csv")